In [1]:
import torch
import random
from seeker import White_seeker, Random_select_seeker, Random_gen_seeker, Black_seeker
from utils import Unfair_metric, load_model, get_L_matrix
from data import adult
from train_dnn import get_data
from dnn_models.model import MLP
from distances import MahalanobisDistances, LogisticRegSensitiveSubspace, BinaryDistance, NormalizedSquaredEuclideanDistance
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [2]:
rand_seed = 0
use_protected_attr = True

dataset, train_dl, test_dl = get_data(adult, rand_seed)
dataset.use_protected_attr = use_protected_attr
in_dim = dataset.dim_feature()
out_dim = 2

model = MLP(in_dim, out_dim)
load_model(model, 'MLP', 'adult', 'STDTrainer', use_protected_attr=use_protected_attr, id=rand_seed)

In [3]:
# prepare data
all_X, all_y = dataset.data, dataset.labels
all_pred = model.get_prediction(all_X)

train_X, train_y = [], []
for x, y in train_dl:
    train_X.append(x)
    train_y.append(y)
train_X = torch.concat(train_X, dim=0)
train_y = torch.concat(train_y, dim=0)

In [4]:
len(all_X)**2*0.0025

5112573.21

In [5]:
adult_gen = adult.Adult_gen(include_protected_feature=use_protected_attr)

In [6]:
# prepare distances
distance_x_Ma = MahalanobisDistances()
distance_x_LR = LogisticRegSensitiveSubspace()
distance_x_NSE = NormalizedSquaredEuclideanDistance()

distance_y = BinaryDistance()

distance_x_Ma.fit(sigma=torch.cov(train_X.T))
distance_x_LR.fit(train_X, protected_idxs=dataset.protected_idxs, keep_protected_idxs=use_protected_attr)
distance_x_NSE.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen)

In [7]:
def rand_gen():
    return {
        'age': random.randint(15, 60),
        'capital_gain': 0,
        'capital_loss': 0,
        'education_num': random.randint(1, 15),
        'hours_per_week': random.randint(10, 50),
        'race_white': random.choice([0, 1]),
        'sex_male': random.choice([0, 1]),
        'marital_status': random.choice(list(range(7))),
        'occupation': random.choice(list(range(14))),
        'relationship': random.choice(list(range(6))),
        'workclass': random.choice(list(range(7)))
    }

def perturb_pair(x, pert, value):
    pair = dict()
    for k, v in x.items():
        pair[k] = torch.tensor([v, v])
        if k == pert:
            pair[k][1] = value
    return pair

x = rand_gen()
pair = adult.generate_from_origin(**perturb_pair(x, 'sex_male', 1-x['sex_male']))
adult.get_original_feature(pair)

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,38.0,0.0,0.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0
1,38.0,0.0,0.0,12.0,27.0,0.0,0.0,4.0,11.0,0.0,1.0


In [8]:
print(1 / distance_x_NSE(pair[0], pair[1]))

tensor([[1.]])


In [9]:
x = rand_gen()
for key in x:
    pair = adult.generate_from_origin(**perturb_pair(x, key, x[key]+1))
    print(distance_x_NSE(pair[0], pair[1]), 1 / distance_x_NSE(pair[0], pair[1]))

tensor([[0.0002]]) tensor([[5328.9941]])
tensor([[1.0000e-10]]) tensor([[9.9998e+09]])
tensor([[5.2702e-08]]) tensor([[18974736.]])
tensor([[0.0044]]) tensor([[225.0000]])
tensor([[0.0001]]) tensor([[9603.9814]])
tensor([[1.]]) tensor([[1.]])
tensor([[1.]]) tensor([[1.]])
tensor([[2.]]) tensor([[0.5000]])
tensor([[2.]]) tensor([[0.5000]])
tensor([[2.]]) tensor([[0.5000]])
tensor([[2.]]) tensor([[0.5000]])


In [10]:
# epsilon越大，要求不公平样本对dx越小，越严格
epsilon = 1
unfair_metric = Unfair_metric(dx=distance_x_NSE, dy=distance_y, epsilon=epsilon)

In [11]:
L = get_L_matrix(all_X, all_pred, distance_x_NSE, distance_y)
n_unfair = torch.sum(L>1/epsilon).item()

total_pairs = all_X.shape[0]**2

unfair_ratio = n_unfair/total_pairs
unfair_ratio

  0%|          | 0/91 [00:00<?, ?it/s]

0.0024952043669610358

In [12]:
1/unfair_ratio

400.7687759932554

In [13]:
p_success = lambda n: 1 - (1 - unfair_ratio)**n
for i in range(10):
    if p_success(10**i) > 1/100:
        print(10**i, p_success(10**i))

10 0.02467372776660104
100 0.22106856817604703
1000 0.9177771946052034
10000 0.9999999999858772
100000 1.0
1000000 1.0
10000000 1.0
100000000 1.0
1000000000 1.0


In [14]:
def show_result(result):
    pair, n_query = result
    if pair != None:
        display(adult.get_original_feature(pair), n_query)
    else:
        print('not found')

In [15]:
# random.seed(422)
# torch.manual_seed(422)

# select_seeker = Random_select_seeker(model=model, unfair_metric=unfair_metric, data=all_X)
# for _ in range(3):
#     show_result(select_seeker.seek(max_query=len(all_X)))

In [16]:
# random.seed(422)
# torch.manual_seed(422)

# gen_seeker = Random_gen_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=True, max_query=len(all_X)))

In [17]:
# random.seed(422)
# torch.manual_seed(422)

# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=False, max_query=len(all_X)))

In [18]:
adult_gen.range

tensor([[1.7000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.0000e+01, 9.9999e+04, 4.3560e+03, 1.6000e+01, 9.9000e+01, 1.0000e+00,
         1.0000e+00, 6.0000e+00, 1.3000e+01, 5.0000e+00, 6.0000e+00]])

In [19]:
random.seed(422)
torch.manual_seed(422)

white_seeker = White_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# show_result(white_seeker.seek())
for _ in range(3):
    show_result(white_seeker.seek(max_query=len(all_X)))

1000000000000000.0
x0 changes
1000000000000000.0
x0 changes
1310.72
out5: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,73.0,0.0,4356.0,1.0,85.0,1.0,0.0,4.0,7.0,1.0,2.0
1,56.0,0.0,4356.0,1.0,70.0,1.0,0.0,4.0,7.0,1.0,2.0


14

1000000000000000.0
x0 changes
1000000000000000.0
x0 changes
1310.72
out5: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,73.0,0.0,4356.0,1.0,85.0,1.0,0.0,4.0,7.0,1.0,2.0
1,56.0,0.0,4356.0,1.0,70.0,1.0,0.0,4.0,7.0,1.0,2.0


14

1000000000000000.0
x0 changes
102400000.0
x0 changes
4096000.0
x0 changes
4096000.0
x0 changes
4096000.0
x0 changes
4096000.0
out5: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,11682.0,0.0,16.0,99.0,1.0,0.0,4.0,7.0,1.0,2.0
1,90.0,8454.0,0.0,16.0,99.0,1.0,0.0,4.0,7.0,1.0,2.0


15

In [20]:
# random.seed(422)
# torch.manual_seed(422)

# black_seeker = Black_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# show_result(black_seeker.seek(max_query=len(all_X)))